In [1]:
import pandas as pd
import ipywidgets as widgets
import datetime as dt
from datetime import datetime
from IPython.display import display, clear_output
from IPython.core.display import HTML
import random
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap, HeatMapWithTime
import tempfile


In [2]:
pd.set_option("display.max_rows", None)

In [3]:
# AVISO SO PROMOTOR E PRACA ATIVA NAO FUNCIONA 


In [4]:
class app:
    def __init__(self, main) -> None:
        self.main = main
        
    def _build_layout(self):
        self.output_main = widgets.Output()
        self.layout = widgets.VBox(children=[self.output_main])
        return self.layout
    
    def _filtrar_pedidos(self, status = None, statusIs = '=='):
        self._atualizar_filtros()
        pedidos = self.main.pedidos.query(
            f"@self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_praca in @self.pracas")
        if status != None:
            pedidos = pedidos.query(f"status {statusIs} @status")
            
        return pedidos
    
    def _filtrar_artigosNoPedido(self, status = "FECHADO"):
        self._atualizar_filtros()
        artigosNoPedido =     self.main.artigosPedido.query(
            f"ref in @self.artigosFiltrar and @self.dataInicial <= ano_mes_cobranca <= @self.dataFinal and id_praca in @self.pracas and status == @status"
        )
        return artigosNoPedido
        
    def _filtrar_desempenho(self):
        self._atualizar_filtros()
        desempenho = self.main.desempenhoPraca.query("@self.dataInicial <= ano_mes_cobranca <= @self.dataFinal  and id_vendedor in @self.promotores and id_praca in @self.pracas")
        return desempenho
    
    def _atualizar_dataframe(self, dataframe, extras = []):
        with self.output_main:
            clear_output()
            dropdown_coluna = widgets.Dropdown(description = 'Campo', options = dataframe.columns)
            dropdown_ascDesc = widgets.Dropdown(description = 'Orientação', options = [('Ascendente', True), ('Descendente', False)])
            button_ordenar = widgets.Button(description = 'Ordenar')
            box_ordem = widgets.HBox(children=[dropdown_coluna, dropdown_ascDesc, button_ordenar])
            button_exportar = widgets.Button(description="Exportar")
            output_df = widgets.Output()
            box_main = widgets.VBox(children=[box_ordem, button_exportar, output_df])
            #TABS
            tab_contents = [widgets.Output() for _ in range(len(extras) + 1)]
            children = [widgets.VBox([tab]) for tab in tab_contents]
            tab = widgets.Tab()
            tab.children = children
            tab.set_title(0, 'DataFrame')
            for i, extra in enumerate(extras):
                tab.set_title(i + 1, extras[i]["nome"])
            display(tab)
            
            
            with tab_contents[0]:
                display(box_main)
            
            with output_df:
                display(dataframe)
                
            for i, extra in enumerate(extras):
                with tab_contents[i + 1]:
                    if isinstance(extra['objeto'], list):
                        for item in extra['objeto']:
                            display(item)
                    elif isinstance(extra['objeto'], folium.Map):
                        display(HTML(f'<div style="width: 1600px; height: 700px;">{extra["objeto"]._repr_html_()}</div>'))
                    else:
                        display(extra['objeto'])

            def atualizar_ordem(s):
                dataframe.sort_values(by= dropdown_coluna.value, ascending = dropdown_ascDesc.value, inplace = True) # type: ignore
                self._atualizar_dataframe(dataframe, extras = extras)
            
            def on_button_click(b):
                agora = datetime.now()
                export = agora.strftime('%Y-%m-%d_%H-%M-%S')
                export_df = dataframe.reset_index()
                export_df.to_excel(f"export/dataframe_exportado_{export}.xlsx", index=True)
            
            button_ordenar.on_click(atualizar_ordem)
            button_exportar.on_click(on_button_click)
        
    def mapaDeBarras(self, df, col):
        df = df.dropna(subset=['latitude', 'longitude'])
        marcadores = []
        altura_maxima = 100
        for index, row in df.iterrows():
            altura_barra = (row[col] / df[col].max()) * altura_maxima
            marcador = folium.Marker(
                location=[row['latitude'], row['longitude']],
                icon=folium.DivIcon(
                    icon_size=(10, altura_barra),
                    icon_anchor=(5, altura_barra / 2),
                    html=f'<div style="width: 10px; height: {altura_barra}px; background-color: blue;"></div>',
                ),
            )
            marcadores.append(marcador)
        
        return marcadores    
    
    def mapaDeCalor(self, df, col ):
        df = df.dropna(subset=['latitude', 'longitude'])
        min = df[col].min()
        max = df[col].max()
        df['scaled'] = (df[col] - min) / (max - min)
        heat_data = [[row['latitude'], row['longitude'], row['scaled']] for index, row in df.iterrows()]
        
        return HeatMap(heat_data)
    
    def criarMapa(self, df, mapsConfigs):
        """_summary_
        Args:
            mapsConfigs (dic): {type = tipo do mapa, col = coluna, kwargs = kwargs}
        """
        df = df.dropna(subset=['latitude', 'longitude'])
        m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=6)
        if isinstance(mapsConfigs, list):
            grupo_marcadores = folium.FeatureGroup(name='Marcadores')  # Crie um FeatureGroup
            for mapConfig in mapsConfigs:
                tipo = getattr(self, mapConfig['type'])
                marcadores = tipo(df, mapConfig['col'])
                if isinstance(marcadores, list):
                    for marcador in marcadores:
                        grupo_marcadores.add_child(marcador)
                else:
                    grupo_marcadores.add_child(marcadores)
            grupo_marcadores.add_to(m)
            # Adicione o controle de camadas após adicionar todos os mapas
            folium.LayerControl().add_to(m)
        else:
            funcao_selecionada = getattr(self, mapsConfigs['type'])
            marcador = funcao_selecionada(df, mapsConfigs['col'])
            if isinstance(marcador, list):
                for marc in marcador:
                    marc.add_to(m)
            else:
                marcador.add_to(m)

        return m
    
    def gerar_cor(self):
        return "#{:02x}{:02x}{:02x}".format(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    
    def _atualizar_filtros(self):
        self.artigosFiltrar = self.main.filtros['artigos']
        self.dataInicial = self.main.filtros['data inicial']
        self.dataFinal = self.main.filtros['data final']
        self.promotores = self.main.filtros['promotores']
        self.pracas = self.main.filtros['pracas']

In [5]:
class cidadesApp(app):
    def __init__(self, main):
        super().__init__(main)        
        self.listaFuncoes = [('Mapa de Praças','posicaoDasPracas'),
                            ('Distribuido', 'distribuicaoAtiva'), 
                            ('Vendido', 'faturamentoCidade'),
                            ('Distribuido X Vendido', 'faturamentoDistribuicao'),
                            ('Vendas (Quantidade)', 'vendasDeArtigos')]
    
    def posicaoDasPracas(self):
        pedidos = self._filtrar_pedidos(status= "DIGITADO")
        df = pedidos.groupby(['cidade', 'descricao_praca']).agg({'id_pedido': 'count', 'latitude': 'mean', 'longitude': 'mean'})
        dfm = df.reset_index()
        mapa = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=10)
        # Itere pelos dados para adicionar marcadores ao mapa
        for _, row in dfm.iterrows():
            praca_info = f"Descrição da Praça: {row['descricao_praca']}<br>Quantidade de Pedidos: {row['id_pedido']}"
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=praca_info,
                icon=folium.Icon(icon='cloud')
            ).add_to(mapa)
        
        self._atualizar_dataframe(df,extras = [{'nome':'Mapa de Calor', 'objeto':mapa}])

        
    def vendasDeArtigos(self):
        artigosNoPedido = self._filtrar_artigosNoPedido(status= "FECHADO")
        mapa_calor = self.criarMapa(artigosNoPedido, {'type': 'mapaDeCalor', 'col':'quantidade_vendida'})        
        artigosNoPedido = artigosNoPedido.groupby('cidade').agg({'id_pedido': 'count','quantidade_vendida': 'sum'})
        self._atualizar_dataframe(artigosNoPedido,extras = [{'nome':'Mapa de Calor', 'objeto':mapa_calor}])

        
    def distribuicaoAtiva(self):
        pedidos = self._filtrar_pedidos(status= "DIGITADO")
        mapa_barras = self.criarMapa(pedidos, {'type': 'mapaDeBarras', 'col':'valor_total_pedido'})
        mapa_calor = self.criarMapa(pedidos, {'type': 'mapaDeCalor', 'col':'valor_total_pedido'})        
        pedidos = pedidos.groupby('cidade').agg({'id_sacoleira': 'count','valor_total_pedido': 'sum'})
        pedidos = pedidos[['id_sacoleira', 'valor_total_pedido']]
        pedidos.rename(columns={'id_sacoleira': 'Pedidos', 'valor_total_pedido':'Valor Distribuido'}, inplace=True)
        pedidos['Sacola Media PV'] = pedidos['Valor Distribuido']/pedidos['Pedidos']
        pedidos = pedidos[['Pedidos', 'Valor Distribuido', 'Sacola Media PV']]
        self._atualizar_dataframe(pedidos,extras = [{'nome':'Mapa de Calor', 'objeto':mapa_calor}])
        
    def faturamentoCidade(self):
        pedidos = self._filtrar_pedidos(status= "FECHADO")
        mapa_barras = self.criarMapa(pedidos, {'type': 'mapaDeBarras', 'col':'valor_venda_pedido'})
        mapa_calor = self.criarMapa(pedidos, {'type': 'mapaDeCalor', 'col':'valor_venda_pedido'})
        pedidos = pedidos.groupby('cidade').agg({'id_sacoleira': 'count','valor_venda_pedido': 'sum'})
        pedidos = pedidos[['id_sacoleira', 'valor_venda_pedido']]
        pedidos.rename(columns={'id_sacoleira': 'Pedidos', 'valor_venda_pedido':'Valor Vendido'}, inplace=True)
        pedidos['Sacola Media PV'] = pedidos['Valor Vendido']/pedidos['Pedidos']
        pedidos = pedidos[['Pedidos', 'Valor Vendido', 'Sacola Media PV']]
        self._atualizar_dataframe(pedidos,extras = [{'nome':'Mapa de Calor', 'objeto':mapa_calor}, {'nome':'Mapa de Barras', 'objeto':mapa_barras}])
        
    def faturamentoDistribuicao(self):
        pedidos = self._filtrar_pedidos(status= "FECHADO")
        mapaTeste = self.criarMapa(pedidos, [{'type': 'mapaDeBarras', 'col': 'valor_total_pedido'}, {'type': 'mapaDeCalor', 'col': 'valor_venda_pedido'}])
        mapa_distribuicao = self.criarMapa(pedidos, {'type': 'mapaDeCalor', 'col':'valor_total_pedido'})
        mapa_venda = self.criarMapa(pedidos, {'type': 'mapaDeCalor', 'col':'valor_venda_pedido'})
        pedidos = pedidos.groupby('cidade').agg({'id_sacoleira': 'count','valor_venda_pedido': 'sum'})
        pedidos = pedidos[['id_sacoleira', 'valor_venda_pedido']]
        pedidos.rename(columns={'id_sacoleira': 'Pedidos', 'valor_venda_pedido':'Valor Vendido'}, inplace=True)
        pedidos['Sacola Media PV'] = pedidos['Valor Vendido']/pedidos['Pedidos']
        pedidos = pedidos[['Pedidos', 'Valor Vendido', 'Sacola Media PV']]
        self._atualizar_dataframe(pedidos,extras = [{'nome': 'Teste', 'objeto': mapaTeste},{'nome':'Mapa de Distribuicao', 'objeto':mapa_distribuicao}, {'nome':'Mapa de Venda', 'objeto':mapa_venda}])

In [6]:
class vendedorasApp(app):
    def __init__(self, main) -> None:
        super().__init__(main)
        self.listaFuncoes = [(('teste', 'teste1'))]

    def _build_layout(self):
        self.output_main = widgets.Output()
        self.layout = widgets.VBox(children=[self.output_main])
        return self.layout
        
    def _atualizar_dataframe(self, dataframe):
        with self.output_main:
            clear_output()
            display(dataframe)
            button_exportar = widgets.Button(description="Exportar")
            display(button_exportar)
            
            def on_button_click(x):
                agora = datetime.now()
                export = agora.strftime('%Y-%m-%d_%H-%M-%S.txt')
                dataframe.to_excel(f"export/dataframe_exportado_{export}.xlsx", index=False)
            button_exportar.on_click(on_button_click)

In [7]:
class financeiroApp(app):
    def __init__(self, main) -> None:
        super().__init__(main)
        self.listaFuncoes = [('Faturamento Periodo', 'faturamento_geral_mes'), 
                            ('Faturamento Promotores', 'faturamento_promotor_mes'), 
                            ('Faturamento Praças','faturamento_praca_mes'),  
                            ('Faturamento Promotor/Praças','faturamento_promotor_praca_mes')]
        
    def faturamento_geral_mes(self):
        desempenho = self._filtrar_desempenho()
        totalFaturado = desempenho[['ano_mes_cobranca','faturamento_liquido']].groupby('ano_mes_cobranca').sum()
        
        fig, ax = plt.subplots(figsize=(10, 6))
        # Plotagem do gráfico de linha
        ax.plot(totalFaturado.index, totalFaturado['faturamento_liquido'], marker='o', linestyle='-')

        # Personalização do gráfico
        ax.set_title('Faturamento Líquido ao Longo do Tempo')
        ax.set_xlabel('Ano e Mês de Cobrança')
        ax.set_ylabel('Faturamento Líquido')
        ax.grid(True)  # Adicionar grades

        # Rotacionar os rótulos do eixo X para melhor legibilidade (opcional)
        plt.xticks(rotation=45)
        plt.tight_layout()  # Ajusta o layout para evitar cortar rótulos
        # Salvar o gráfico em uma variável
        meu_grafico = plt.gcf()

        plt.close()

        self._atualizar_dataframe(totalFaturado, extras = [{'nome':'grafico', 'objeto':meu_grafico}] )
        
    def faturamento_promotor_mes(self):
        desempenho = self._filtrar_desempenho()
        totalFaturado = desempenho[['ano_mes_cobranca','faturamento_liquido', 'descricao_vendedor']].groupby(['ano_mes_cobranca', 'descricao_vendedor']).sum()
        totalFaturado = totalFaturado.sort_values(by= ['ano_mes_cobranca', 'faturamento_liquido'], ascending= False)
        # Pivotar os dados para ter os vendedores como colunas
        pivot_df = totalFaturado.pivot_table(index='ano_mes_cobranca', columns='descricao_vendedor', values='faturamento_liquido', fill_value=0)

        # Calcular os totais de cada vendedor e ordenar as colunas com base nos totais
        col_order = pivot_df.sum().sort_values(ascending=False).index
        pivot_df = pivot_df[col_order]
        # Crie uma figura e um eixo
        fig, ax = plt.subplots(figsize=(12, 6))

        # Gere cores aleatórias para os vendedores
        num_vendedores = len(pivot_df.columns)
        colors = ['#%02x%02x%02x' % (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for _ in range(num_vendedores)]

        # Plotagem do gráfico de barras empilhadas
        bottom = None
        for vendedor, color in zip(pivot_df.columns, colors):
            ax.bar(
                pivot_df.index,
                pivot_df[vendedor],
                label=vendedor,
                bottom=bottom, # type: ignore
                color=color,
                alpha=1,
                width=10.0,  # Ajuste a largura das barras conforme desejado
            )
            if bottom is None:
                bottom = pivot_df[vendedor]
            else:
                bottom += pivot_df[vendedor]


        # Personalização do gráfico
        ax.set_title('Faturamento Líquido por Vendedor ao Longo do Tempo')
        ax.set_xlabel('Ano e Mês de Cobrança')
        ax.set_ylabel('Faturamento Líquido')
        ax.grid(True)  # Adicionar grades

        # Posicionar a legenda abaixo do gráfico
        ax.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0), ncol=1)  # Mude 'upper left' para 'upper right' se preferir à direita

        # Rotacionar os rótulos do eixo X para melhor legibilidade (opcional)
        plt.xticks(rotation=45)
        
        meu_grafico = plt.gcf()
        plt.close()
        self._atualizar_dataframe(totalFaturado, extras = [{'nome':'grafico', 'objeto':meu_grafico}] )        
    
    def faturamento_praca_mes(self):
        desempenho = self._filtrar_desempenho()
        totalFaturado = desempenho[['ano_mes_cobranca','faturamento_liquido', 'descricao_praca']].groupby(['ano_mes_cobranca', 'descricao_praca']).sum()
        self._atualizar_dataframe(totalFaturado)
        
    def faturamento_promotor_praca_mes(self):
        desempenho = self._filtrar_desempenho()
        totalFaturado = desempenho[['ano_mes_cobranca','faturamento_liquido','descricao_vendedor', 'descricao_praca']].groupby(['ano_mes_cobranca', 'descricao_vendedor', 'descricao_praca']).sum()
        self._atualizar_dataframe(totalFaturado)

In [8]:
class artigosApp(app):
    def __init__(self, main):
        super().__init__(main)        
        self.listaFuncoes = [('Deixado e Vendido Simples', 'deixado_vendido_simples'), 
                            ('Deixado e Vendido (Promotor/Mês)' ,'deixado_vendido_mensal_promotor'),
                            ('Deixado e Vendido (Promotor/Periodo)' ,'deixado_vendido_periodo_promotor'),
                            ('Deixado, Vendido, Na rua (Geral)', 'deixado_vendido_simples_digitado'),
                            ('Deixado, Vendido, Na rua (Promotores)','deixado_vendido_simples_digitado_promotor'),
                            ('Deixado, Vendido e Faturado', 'deixado_vendido_faturado'),
                            ('Deixado, Vendido por mês', 'deixado_vendido_mensal'),
                            ('Deixado, Vendido, Rua e Score', 'deixado_vendido_rua_score'),
                            ('Variações por periodo','variacao_periodo'),
                            ('Matriz BCG', 'bcg_periodo')
                            ]
        
    def deixado_vendido_simples(self):
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df = df.reset_index()
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        ordem = ['ref', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida']
        df = df.reindex(columns=ordem)
        df.sort_values(by='artigo', inplace= True)
        self._atualizar_dataframe(df)
    
    def deixado_vendido_simples_digitado(self):
        artigosNoPedido = self._filtrar_artigosNoPedido(status="FECHADO")
        totalDigitado = self._filtrar_artigosNoPedido(status= "DIGITADO")
        totalDigitado = totalDigitado.groupby(['ref', 'artigo']).agg({'quantidade_entregue':'sum'})
        totalDigitado.rename(columns={'quantidade_entregue': 'Na Rua'}, inplace=True)
        df = artigosNoPedido.groupby(['ref', 'artigo']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df = pd.merge(df, totalDigitado[['Na Rua']], on=['ref', 'artigo'])
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        self._atualizar_dataframe(df)
        
    def deixado_vendido_simples_digitado_promotor(self):
        artigosNoPedido = self._filtrar_artigosNoPedido(status="FECHADO")
        totalDigitado = self._filtrar_artigosNoPedido(status= "DIGITADO")
        totalDigitado = totalDigitado.groupby(['ref', 'artigo', 'descricao_vendedor']).agg({'quantidade_entregue':'sum'})
        totalDigitado.rename(columns={'quantidade_entregue': 'Na Rua'}, inplace=True)
        df = artigosNoPedido.groupby(['ref', 'artigo', 'descricao_vendedor']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df = pd.merge(df, totalDigitado[['Na Rua']], on=['ref', 'artigo', 'descricao_vendedor'])
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        self._atualizar_dataframe(df)
        
    def deixado_vendido_mensal(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'ano_mes_cobranca']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        ordem = ['ref', 'ano_mes_cobranca', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida']
        self._atualizar_dataframe(df)
        
    def deixado_vendido_mensal_promotor(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'descricao_vendedor', 'ano_mes_cobranca']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        ordem = ['ref', 'ano_mes_cobranca', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida']
        self._atualizar_dataframe(df)
        
    def deixado_vendido_periodo_promotor(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo', 'descricao_vendedor']).agg({'quantidade_entregue':'sum', 'quantidade_vendida':'sum'})
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        ordem = ['ref', 'ano_mes_cobranca', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida']
        self._atualizar_dataframe(df)
    
    def deixado_vendido_rua_score(self):
        self._atualizar_filtros()
        artigosNoPedido = self._filtrar_artigosNoPedido()
        totalDigitado = self._filtrar_artigosNoPedido(status="DIGITADO")[['ref', 'quantidade_entregue']]
        totalDigitado = totalDigitado.groupby('ref').sum().reset_index()
        totalDigitado.rename(columns={'quantidade_entregue': 'Na Rua'}, inplace=True)
        df = artigosNoPedido.groupby('ref').sum()
        df = pd.merge(df, totalDigitado[['ref', 'Na Rua']], on='ref')
        df['% Vendida'] = df['quantidade_vendida'] / df['quantidade_entregue']
        df['Score'] = df['quantidade_vendida'] * df['% Vendida']
        ordem = ['ref', 'artigo', 'quantidade_entregue', 'quantidade_vendida', '% Vendida', 'Na Rua','Score']
        df = df.reindex(columns=ordem)
        df = df.sort_values('Score', ascending= False)
        self._atualizar_dataframe(df)
    
    def deixado_vendido_faturado(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'artigo']).sum()
        ordem = ['quantidade_entregue', 'quantidade_vendida', 'valor_vendido']
        df = df.reindex(columns=ordem)
        df = df.sort_values('valor_vendido', ascending= False)
        self._atualizar_dataframe(df)
        
    def variacao_periodo(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'ano_mes_cobranca']).sum().reset_index()
        df['giro'] = df['ano_mes_cobranca']
        meses = df['ano_mes_cobranca'].unique()
        grupos = [meses[i:i+2] for i in range(0, len(meses), 2)]
        giro = 0
        for grupo in grupos:
            giro += 1
            for data in grupo:
                df['giro'] = df['giro'].replace(data, giro)
                
        df = df.groupby(['ref', 'giro']).sum().reset_index()
        
        df['flutuacao_entregue'] = df.groupby('ref')['quantidade_entregue'].pct_change() * 100
        df['mudanca_total_entregue'] = df.groupby('ref')['flutuacao_entregue'].cumsum()
        df['flutuacao_vendida'] = df.groupby('ref')['quantidade_vendida'].pct_change() * 100
        df['mudanca_total_vendida'] = df.groupby('ref')['flutuacao_vendida'].cumsum()
        df['porcentagem_vendida'] = df['quantidade_vendida'] / df['quantidade_entregue'] * 100
        df['flutuacao_porcentagem_vendida'] = df.groupby('ref')['porcentagem_vendida'].pct_change() * 100
        df['mudanca_total_porcentagem_vendida'] = df.groupby('ref')['flutuacao_porcentagem_vendida'].cumsum()
        
        ordem = ['ref', 'quantidade_entregue','flutuacao_entregue', 'mudanca_total_entregue', 'quantidade_vendida', 'flutuacao_vendida',  'mudanca_total_vendida', 'porcentagem_vendida', 'flutuacao_porcentagem_vendida','mudanca_total_porcentagem_vendida']
        df = df.reindex(columns=ordem)

        nomes = {'quantidade_entregue': 'Entregue', 'quantidade_vendida': 'Vendida', 'flutuacao_entregue' : 'Mov. Entregue', 'mudanca_total_entregue': 'Mov.To Entregue', 'flutuacao_vendida': 'Mov. Vendida', 'mudanca_total_vendida': 'Mov.To Vendida', 'porcentagem_vendida': '% Vendida', 'flutuacao_porcentagem_vendida': 'Mov. % Vendida','mudanca_total_porcentagem_vendida': 'Mov.To % Vendida' }
        df.rename(columns= nomes, inplace = True)
        self._atualizar_dataframe(df)
        
    def bcg_periodo(self):
        self._atualizar_filtros()
        df = self._filtrar_artigosNoPedido()
        df = df.groupby(['ref', 'ano_mes_cobranca']).sum().reset_index()
        df['giro'] = df['ano_mes_cobranca']
        meses = df['ano_mes_cobranca'].unique()
        grupos = [meses[i:i+2] for i in range(0, len(meses), 2)]
        giro = 0
        for grupo in grupos:
            giro += 1
            for data in grupo:
                df['giro'] = df['giro'].replace(data, giro)
                
        df = df.groupby(['ref', 'giro']).sum().reset_index()
        
        df['Taxa_Crescimento'] = df['quantidade_vendida'].pct_change() * 100
        # Calcular a Participação de Mercado Relativa
        df['Participacao_Mercado'] = (df['quantidade_vendida'] / df['quantidade_vendida'].sum()) * 100
        
        # Classificar os produtos/unidades de negócios
        def classify(row):
            if row['Taxa_Crescimento'] > 0 and row['Participacao_Mercado'] > 0:
                return 'Estrela'
            elif row['Taxa_Crescimento'] > 0 and row['Participacao_Mercado'] <= 0:
                return 'Interrogação'
            elif row['Taxa_Crescimento'] <= 0 and row['Participacao_Mercado'] > 0:
                return 'Vaca Leiteira'
            else:
                return 'Abacaxi'
        df['Classificacao'] = df.apply(classify, axis=1)
        
        # Criar o gráfico de dispersão
        fig, ax = plt.subplots(figsize=(10, 6))
        
        ax.scatter(df['Participacao_Mercado'], df['Taxa_Crescimento'], c='blue', marker='o') # type: ignore
        ax.set_xlabel('Participação de Mercado Relativa (%)')
        ax.set_ylabel('Taxa de Crescimento do Mercado (%)')

        for i, row in df.iterrows():
            ax.annotate(row['Classificacao'], (row['Participacao_Mercado'], row['Taxa_Crescimento']))

        ax.set_title('Matriz BCG')
        ax.grid(True)

        for i, row in df.iterrows():
            plt.annotate(row['Classificacao'], (row['Participacao_Mercado'], row['Taxa_Crescimento']))
        
        plt.title('Matriz BCG')
        plt.grid(True)
        plt.close()
        self._atualizar_dataframe(df, extras = [{'nome':'grafico', 'objeto':fig}])
        

In [9]:
class MainApp:
    def __init__(self):
        self.app_artigos = artigosApp(self)
        self.app_vendedoras = vendedorasApp(self)
        self.app_financeiro = financeiroApp(self)
        self.app_cidades = cidadesApp(self)
        self.apps = [
            ('Artigos', self.app_artigos),
            ('Financeiro', self.app_financeiro), 
            ('Cidades', self.app_cidades)]
        self.load_data()
        self.create_layout() 
        self.rodarApp()
        
    def load_data(self):
        self.vendedores = pd.read_csv('data/vendedores.csv', sep=';')
        self.pracas = pd.read_csv('data/pracas.csv', sep=';')
        self.desempenhoPraca = pd.read_csv('data/desempenhoPraca.csv', sep=';')
        self.pedidos = pd.read_csv('data/pedidos.csv', sep=';')
        self.sacoleiras = pd.read_csv('data/sacoleiras.csv', sep=';')
        self.artigosPedido = pd.read_csv('data/artigosPedido.csv', sep=';')
        self.gruposArtigos = pd.read_csv('data/grupos.csv', sep=';')
        self.artigos = pd.read_csv('data/artigos.csv', sep=';')
        self.infoCidades = pd.read_csv('data/cidades.csv', sep=';')
        
        self.artigosPedido['ano_mes_cobranca'] = pd.to_datetime(self.artigosPedido['ano_mes_cobranca'], format='%m/%Y').dt.date
        self.desempenhoPraca['ano_mes_cobranca'] = pd.to_datetime(self.desempenhoPraca['ano_mes_cobranca'], format='%Y-%m').dt.date
        self.pedidos['ano_mes_cobranca'] = pd.to_datetime(self.pedidos['ano_mes_cobranca'], format='%m/%Y').dt.date

    def create_layout(self):
        # Promotor
        self.check_soPromotorEPracaAtivo = widgets.Checkbox(description="Só Ativas?", value=True)
        listaPromotores = self.get_lista_promotores()
        self.select_promotores = widgets.SelectMultiple(description='Promotor(es)', options=listaPromotores, value=[promotor[1] for promotor in listaPromotores])
        self.check_todosPromotores = widgets.Checkbox(description="Todos", value=False)
        self.box_promotores = widgets.HBox(children=[self.select_promotores, self.check_todosPromotores])
        
        # Praça
        listaPracas = self.get_lista_pracas()
        self.select_pracas = widgets.SelectMultiple(description='Praça(s)', options=listaPracas, value=[praca[1] for praca in listaPracas])
        self.check_todasPracas = widgets.Checkbox(description="Todas", value=False)
        self.box_pracas = widgets.HBox(children=[self.select_pracas, self.check_todasPracas])
        self.box_promotorPraca = widgets.HBox(children=[self.box_promotores, self.box_pracas, self.check_soPromotorEPracaAtivo])
        
        # Periodo
        self.dateP_dataInicial = widgets.DatePicker(description='Data Inicial Cobrança', value=dt.date.today() - dt.timedelta(days=2 * 30))
        self.dateP_dataFinal = widgets.DatePicker(description='Data Final Cobrança', value=dt.date.today() + dt.timedelta(days=2 * 30))
        self.box_periodo = widgets.HBox(children=[self.dateP_dataInicial, self.dateP_dataFinal])
        
        # Grupos e Artigos
        listaGrupos = self.get_lista_grupos()
        self.select_gruposArtigos = widgets.SelectMultiple(description='Grupos', options=listaGrupos, value=[grupo[1] for grupo in listaGrupos])
        self.check_todosGrupos = widgets.Checkbox(description='Todos', value=False)
        listaArtigos = self.get_lista_artigos()
        self.select_artigos = widgets.SelectMultiple(description='Artigos', options=listaArtigos, value=[artigo[1] for artigo in listaArtigos])
        self.check_todosArtigos = widgets.Checkbox(description='Todos', value=False)
        self.box_gruposArtigos = widgets.HBox(children=[self.select_gruposArtigos, self.check_todosGrupos, self.select_artigos, self.check_todosArtigos])

        # Relatórios
        self.select_app = widgets.Dropdown(description='App:', options=self.apps)
        self.select_funcao = widgets.Dropdown(description='Função:', options=self.get_lista_funcoes())
        self.togle_modoExibicao = widgets.ToggleButtons(
            options=['Mesma Tab', 'Nova Tab'],
            description='Relatório na:',
            value='Mesma Tab',
            tooltips=['O relatório substituirá a tab atual', 'O relatório entrará em uma nova tab'],
        )
        self.button_rodar = widgets.Button(description= 'Rodar')
        self.button_rodar.on_click(self._app_selecionado)
        self.box_apps = widgets.HBox(children=[self.select_app, self.select_funcao, self.button_rodar])

        self.filtros = [self.box_promotorPraca, self.box_periodo, self.box_gruposArtigos]
        self.box_filtros = widgets.VBox(children=self.filtros)
        self.display_app = widgets.HBox(children=[widgets.Output()])
        self.tab_apps = widgets.Tab(children=[self.display_app])
        self.box_fixos = [self.box_filtros, self.box_apps, self.tab_apps]
        self.box_main = widgets.VBox(children=self.box_fixos)

        # Add observers to all widgets
        self.add_observer_to_widgets(self.box_filtros)

        # Display the nested boxes
        display(self.box_main)
    
    def get_lista_funcoes(self):
        # return [func for func in dir(self.select_app.value) if callable(getattr(self.select_app.value, func)) and not func.startswith("__") and # not func.startswith("_")]
        return self.select_app.value.listaFuncoes # type: ignore

    def get_lista_promotores(self):
        listaPromotores = []
        promotores = self.vendedores.sort_values(by='descricao_vendedor', ascending=True)
        if self.check_soPromotorEPracaAtivo.value:
            promotores = promotores.query('status_vendedor == "ATIVO"')

        for _, linha in promotores.iterrows():
            tupla = (linha['descricao_vendedor'], linha['id_vendedor'])
            listaPromotores.append(tupla)

        return listaPromotores

    def get_lista_pracas(self):
        listaPracas = []
        promotores = self.select_promotores.value
        pracas = self.pracas.query('id_vendedor.isin(@promotores)')
        pracas = pracas.sort_values(by='descricao_praca', ascending=True)

        if self.check_soPromotorEPracaAtivo.value:
            pracas = pracas.query('status_praca == "ATIVO"')

        for _, linha in pracas.iterrows():
            tupla = (linha['descricao_praca'], linha['id_praca'])
            listaPracas.append(tupla)

        return listaPracas

    def get_lista_grupos(self):
        listaGrupos = []
        grupos = self.gruposArtigos
        grupos = grupos.sort_values(by='descricao_grupo', ascending=True)

        for _, linha in grupos.iterrows():
            tupla = (linha['descricao_grupo'], linha['id_grupo'])
            listaGrupos.append(tupla)

        return listaGrupos

    def get_lista_artigos(self):
        listaArtigos = []
        grupos = self.select_gruposArtigos.value
        artigos = self.artigos.query('id_grupo.isin(@grupos)')
        artigos = artigos.sort_values(by='artigo', ascending=True)

        if self.check_soPromotorEPracaAtivo.value:
            artigos = artigos.query('status_artigo == "ATIVO"')

        for _, linha in artigos.iterrows():
            tupla = (linha['artigo'], linha['ref'])
            listaArtigos.append(tupla)

        return listaArtigos

    def add_observer_to_widgets(self, widget):
        # Specific functions
        self.select_promotores.observe(self._promotor_selecionado, names='value') # type: ignore
        self.select_gruposArtigos.observe(self._grupos_selecionados, names='value') # type: ignore
        self.check_todosPromotores.observe(self._selecionar_todos_promotores, names='value') # type: ignore
        self.check_todasPracas.observe(self._selecionar_todas_pracas, names='value') # type: ignore
        self.check_todosGrupos.observe(self._selecionar_todos_grupos, names='value') # type: ignore
        self.check_todosArtigos.observe(self._selecionar_todos_artigos, names='value') # type: ignore
        self.select_app.observe(self._app_selecionado, names='value') # type: ignore
        self.select_funcao.observe(self._funcao_selecionada, names='value') # type: ignore

        # General function
        if isinstance(widget, (widgets.Box, widgets.VBox, widgets.HBox, widgets.GridBox)):
            for child_widget in widget.children: # type: ignore
                self.add_observer_to_widgets(child_widget)
        elif isinstance(widget, widgets.Widget):
            widget.observe(self.normal_object_updated, names='value') # type: ignore

    def normal_object_updated(self, change):
        if change['new'] == ():
            pass
        else:
            self._update_termos()
                
    def _update_termos(self):
        # Convertendo as strings das datas para o formato "mm/aaaa"
        data_inicial_str = self.dateP_dataInicial.value.strftime("%m/%Y") # type: ignore
        data_final_str = self.dateP_dataFinal.value.strftime("%m/%Y") # type: ignore

        # Convertendo as strings para o tipo datetime
        data_inicial = datetime.strptime(data_inicial_str, "%m/%Y").date()
        data_final = datetime.strptime(data_final_str, "%m/%Y").date()

        self.filtros = {
            'promotores': self.select_promotores.value,
            'pracas': self.select_pracas.value,
            'data inicial': data_inicial,
            'data final': data_final,
            'artigos': self.select_artigos.value
        }
        
    def set_funcao(self):
        funcao_selecionada = getattr(self.select_app.value, self.select_funcao.value) # type: ignore
        funcao_selecionada()

    def _updateOptionsPracas(self):
        self.select_pracas.options = self.get_lista_pracas()
        self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]

    def _updateOptionsArtigos(self):
        self.select_artigos.options = self.get_lista_artigos()
        self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]

    def _updateOptionsFuncoes(self):
        self.select_funcao.options = self.get_lista_funcoes()

    def _promotor_selecionado(self, change):
        self._updateOptionsPracas()

    def _grupos_selecionados(self, change):
        self._updateOptionsArtigos()

    def _funcao_selecionada(self, change):
        self._update_termos()
        #self.set_funcao()

    def _app_selecionado(self, change):
        self._updateOptionsFuncoes()
        self.rodarApp()
        self.set_funcao()
        
    def rodarApp(self):
        selected_app = self.select_app.value
        app_widgets = selected_app._build_layout() # type: ignore
        self.display_app.children = [app_widgets]
        # app_widgets = [app_instance._build_layout() for _, app_instance in self.apps]
        # self.display_app.children = app_widgets

    def _selecionar_todos_promotores(self, change):
        if change['new']:
            self.select_promotores.value = [promotor[1] for promotor in self.get_lista_promotores()]
        else:
            self.select_promotores.value = []

    def _selecionar_todas_pracas(self, change):
        if change['new']:
            self.select_pracas.value = [praca[1] for praca in self.get_lista_pracas()]
        else:
            self.select_pracas.value = []

    def _selecionar_todos_grupos(self, change):
        if change['new']:
            self.select_gruposArtigos.value = [grupo[1] for grupo in self.get_lista_grupos()]
        else:
            self.select_gruposArtigos.value = []

    def _selecionar_todos_artigos(self, change):
        if change['new']:
            self.select_artigos.value = [artigo[1] for artigo in self.get_lista_artigos()]
        else:
            self.select_artigos.value = []


In [10]:
app = MainApp()